## CRAG : Corrective RAG

CRAG 전략을 사용하여 RAG 기반 시스템을 개선하는 방법을 다룹니다.  
CRAG는 검색된 문서들에 자기에 대한 `자기반성 (self-reflection)` 및  `자기 평가 (self-evaluation)` 단계를 포함하여, 검색-생성 파이프라인을 정교하게 다루는 접금법 입니다  

--------------------------------------------------------

#### CRAG란? 
**Corrective-RAG**는 RAG(Retrieval Augmented Generation) 전략에서 **검색 과정에서 찾아온 문서를 평가하고, 지식을 정제 (refine) 하는 단계를 추가한 방법론입니다. 이는 생성에 앞서 검색 결과를 점검하고 필요하다면 보조적인 검색을 수행하며, 최종적으로 품질 높은 답변을 생성하기 위한 일련의 프로세스를 포함합니다.

CRAG의 핵심 아이디어는 다음과 같습니다.   

[논문(Corrective Retrieval Augmented Generation)](https://arxiv.org/pdf/2401.15884)  
1. 검색된 문서 중 하나 이상이 사전 정의된 관련성 임계값 (retrieval validation score) 을 초과하면 생성단계로 진행합니다.  
2. 생성 전에 지식 정제 단계를 수행합니다.  
3. 문서를 "knowledge strips" 로 세분화합니다. (여기서, 문서 검색 결과수, `k`를 의미합니다)
4. 모든 문서가 관련성 임계값 이하이거나 평가 결과 신뢰도가 낮을 경우, 추가 데이터 소스 (예: 웹검색) 로 보강합니다. 
5. 모든 문서가 관련성 임계값 이하이거나 평가 결과 신뢰도가 낮을 경우, 추가 데이터 소스 (예: 웹검색)로 보강합니다. 
6. 웹 검색을 통한 보강 시, 쿼리 재작성 (Query-Rewrite) 을 통해 검색 결과를 최적화합니다. 

--------------------------------------------------------

#### 주요 내용
LangGraph를 활용하여 CRAG 접근법의 일부 아이디어를 구현합니다.  
여기서는 **지식 정제 단계는 생략** 하고, 필요하다면 노드로 추가할 수 있는 형태로 설계합니다.  
또한, **관련 있는 문서가 하나도 없으면** 웹 검색을 활용하여 검색을 보완할 것입니다.
웹 검색에는 **Tavily Search**를 사용하고, 검색 최적화를 위해 질문 재작성(Question Re-writing)을 도입합니다.  

--------------------------------------------------------

#### 주요 
* **Retrieval Grader** : 검색된 문서의 관련성을 평가  
* **Generate** : LLM 을 통한 답변 생성  
* **Question Re-writer** : 질문 재작성을 통한 검색 질의 최적화  
* **Web Search Tool** : Tavily Search 를 통한 웹검색 활용 
* **Create Graph** : LangGraph 를 통한 CRAG 전략 그래프 생성 
* **Use the graph** : 생성된 그래프를 활용하는 방법 

--------------------------------------------------------

#### 참고 
* [LangGraph CRAG 튜토리얼 (공식 문서)](https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_crag_local/)


In [ ]:
# 환경설정 
from dotenv import load_dotenv
load_dotenv()

from config.langsmith import langsmith 
langsmith.set_project("langgraph-crag")

: 

## 기본 PDF 기반 Retrieval Chain 생성 

PDF 문서를 기반으로 Retrieval Chain 을 생성합니다. 가장 단순한 구조의 Retrieval Chain입니다.   
단, LangGraph에서는 Retriever와 Chain을 따로 생성합니다. 그래야 각 노드별로 세부 처리를 할 수 있습니다.  

**실습에 활용한 문서** 
소프트웨어정책연구소(SPRi) - 2023년 12월호
* 저자: 유재흥(AI정책연구실 책임연구원), 이지수(AI정책연구실 위촉연구원)
* 링크: https://spri.kr/posts/view/23669
* 파일명: SPRI_AI_Brief_2023년12월호_F.pdf

In [ ]:
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI

# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()
print(f"문서 페이지 수 : {len(docs)}")

print(docs[0].page_content)

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")


# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")

# 단계 3: 임베딩(Embedding) 생성
# embeddings = OpenAIEmbeddings()
embeddings = GoogleGenerativeAIEmbeddings()

# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
# FAISS (Facebook AI Similarity Search)는 밀집 벡터의 효율적인 유사도 검색과 클러스터링을 위한 라이브러리
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.
# retriever.invoke("삼성전자가 자체 개발한 AI 의 이름은?")

# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """
    You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know. 
    Answer in Korean.

    #Context: 
    {context}

    #Question:
    {question}

    #Answer:
    """
)
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatGoogleGenerativeAI(model_name="gemini-2.0-flash", temperature=0)

# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 단계 9: 체인 실행(Run Chain)
# 체인을 실행합니다.
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "삼성전자가 자체 개발한 AI 의 이름은?"
response = chain.invoke(question)
print(response)

## 검색된 문서의 관련성 평가 (Question-Retrieval Evaluation)

검색된 문서의 관련성 평가는 검색된 문서가 질문과 관련이 있는지 여부를 평가하는 단계입니다.  
먼저, 검색된 문서를 평가하기 위한 `평가기(retrieval-grader)` 를 생성합니다.

In [ ]:
import os 
from langchain_core.prompts import ChatPromptTemplate 
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field 

# 검색된 문서의 관련성 여부를 이진 점수로 평가하는 데이터 모델 
class GradeDocuments(BaseModel):
    """ Binary score to determine the relevance of the retrieved document"""

    # 문서가 질문과 관련이 있는지 여부를 'yes' 또는 'no' 로 나타내는 필드 
    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    api_key=os.getenv("GOOGLE_API_KEY")
)

# GradeDocuments 데이터 모델을 사용하여 구조화된 출력을 생성 
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# 시스템 프롬프트 정의
system = """ 
    You are a grader assessing relevance of a retrieved document to a user question. 
    If the document contains keywords or semantic meaning related to the question, grade it as relevant.
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question.
"""

# 채팅 프롬프트 탬플릿 생성
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}")
    ]
)

# Retrieval 평가기 초기화
retrieval_grader = grade_prompt | structured_llm_grader 

`retrieval_grader`를 사용해서 문서를 평가합니다.   
여기서는 문서의 집합이 아닌 1개의 단일 문서에 대한 평가를 수행합니다.  
결과는 단일 문서에 대한 관련성 여부가 (yes / no) 로 반환됩니다.

In [ ]:
# 질문 정의 
question = "삼성전자가 개발한 생성AI에 대해 설명하세요"

# 문서 검색 
docs = retriever.invoke(question)

# 문서 평가 
# 검색된 문서 중 0번째 index 문서의 페이지 내용 추출 
doc_text = docs[0].page_content 

# 검색된 문서와 질문을 사용해서 관련성 평가를 실행하고 결과 출력 
print(retrieval_grader.invoke({"question":question, "document":doc_text}))





## 답변 생성 체인 

답변 생성 체인은 검색된 문서를 기반으로 답변을 생성하는 체인입니다.   
우리가 알고 있는 일반적인 Naive RAG 체인입니다. 

In [ ]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser 
from langchian_google_genai import ChatGoogleGenerativeAI 

rag_prompt_system = """ 
 You are an AI assistant that can answer questions about the following context:
 Your task is answering questions based on the context provided. 
 If you cannot find the answer in the context, just say "I cannot find the answer in the context".
 You should answer in Korean. 
 Answer in English for name and technical terms. 
"""

rag_prompt_human = """ 
# question: {question}
# context: {context}
# answer: 
"""

rag_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", rag_prompt_system),
        ("human", rag_prompt_human),
    ]
)

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    api_key=os.getenv("GOOGLE_API_KEY")
)

rag_chain = rag_prompt_template | llm | StrOutputParser()

def format_docs(docs):
    return "\n\n".join(
        [
            f'<document><content>{doc.page_content}</content><source>{doc.metadata["source"]}</source><page>{doc.metadata["page"]+1}</page></document>'
            for doc in docs
        ]
    )

generation = rag_chain.invoke({"context" : format_docs(docs), "question" : question})
print(generation)

## 쿼리 재작성 (Question Re-writer)

쿼리 재작성은 웹 검색을 최적화하기 위해 질문을 재작성하는 단계입니다. 

In [ ]:
# 쿼리 재작성 (Question Re-writer) 시스템 프롬프트
system_prompt = """ 
    You are a question re-writer that converts an input question to a better version that is optimized for web search. 
    Look at the input and try to reason about the underlying semantic intent / meaning
"""

# 프롬프트 정의
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "Here is the initial question: \n\n {question} \n Formulate an improved question.")
    ]
)

# Question Re-writer 체인 초기화 
re_write_chain = re_write_prompt | llm | StrOutputParser()

print(f"[원본 질문] : {question}")
print(f"[재작성 질문] :", re_write_chain.invoke({"question": question}))

## 웹 검색 도구 
**웹 검색 도구**는 컨텍스트를 보강하기 위한 용도로 사용됩니다. 
* **웹 검색의 필요성** : 모든 문서가 관련성 임계값을 충족하지 않거나 평가자가 확신이 없을 때, 웹 검색을 통해 추가 데이터를 확보합니다. 
* **Tavily Search 사용** : Tavily Search를 활용하여 웹검색을 수행합니다. 이는 검색 쿼리를 최적화 하고, 보다 관련성 높은 결과를 제공합니다. 
* **질문 재작성** : 웹 검색을 최적화하기 위해 질문을 재작성하여 검색 쿼리를 개선합니다. 

In [ ]:
# 웹 검색 도구 초기화
from config.tavily_search import TavilySearch 

# 최대 검색 결과를 3으로 설정
web_search_tool = TavilySearch(max_results=3)

results = web_search_tool.invoke({"query":question})
print(results)

: 

## 상태 (State)

CRAG 그래프를 위한 상태를 정의합니다.   
`web_search`는 웹 검색을 활용할지 여부를 나타내는 상태입니다.  
**yes** 또는 **no** 로 표현합니다 (yes : 웹 검색 필요, no : 필요없음)

In [ ]:
from typing import Annotated, List 
from typing_extensions import TypedDict 

# 상태정의 
class GraphState(TypedDict):
    question : Annotated[str, "The question to answer"]
    generation : Annotated[str, "The generation from the LLM"]
    web_search : Annotated[str, "whether to add search"]
    documents: Annotated[List[str], "The documents retrieved from the vector store"]

## 노드 

CRAG 그래프에 활용할 노드를 정의합니다. 

In [ ]:
from langchain.schema import Document 

# 문서 검색 노드
def retrieve(state: GraphState):
    print("===Retrieve===")
    question = state["question"]

    # 문서 검색 
    documents = retriever.invoke(question)
    return {"documents": documents}

# 답변 생성 노드
def generate(state: GraphState):
    print("===Generate===")
    question = state["question"]
    documents = state["documents"]

    # RAG를 사용한 답변 생성 
    generation = rag_chain.invoke({"context": documents, "question":question})
    return {"generation": generation}

# 문서 평가 노드 
def grade_documents(state: GraphState):
    print("===Grade Documents===")
    question = state["question"]
    documents = state["documents"]

    # 필터링된 문서 
    filtered_docs = [] 
    relevant_doc_count = 0

    for doc in documents:
        # Question - Document 관련성 평가
        score = retrieval_grader.invoke(
            {"question": question, "document": doc.page_content}
        )
        grade = score.binary_score

        if grade == "yes":
            print("=== [GRADE : DOCUMENT RELEVANT] ===")
            # 관련 있는 문서를 filtered_docs에 추가 
            filtered_docs.append(doc)
            relevant_doc_count += 1
        else: 
            print("=== [GRADE : DOCUMENT IRRELEVANT] ===")
            continue 

    # 관련 문서가 없으면 웹 검색 수행
    web_search = "yes" if relevant_doc_count == 0 else "no"
    return {"documents": filtered_docs, "web_search": web_search}

# 쿼리 재작성 노드
def query_rewrite(state: GraphState):
    print("===Query Rewrite===")
    question = state["question"]

    # 질문 재작성
    better_question = re_write_chain.invoke({"question": question})
    return {"question": better_question}

# 웹 검색 노드
def web_search(state: GraphState):
    print("===Web Search===")
    question = state["question"]
    documents = state["documents"]

    # 웹 검색 실행
    docs = web_search_tool.invoke({"query": question})

    # 검색 결과를 문서 형식으로 변환 
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    documents.append(web_results)
    
    return {"documents": documents}

## 조건부 엣지에 활용할 함수 
`decide_to_generate` 함수는 관련성 평가를 마친 뒤, 웹 검색 여부에 따라 다음 노드로 라우팅하는 역할을 수행합니다.    
`web_search`가 `YES`인 경우 `query_rewirte` 노드에서 쿼리를 재작성 한 뒤 웹 검색을 수행합니다.   
만약, `web_search`가 `No`인 경우는 `generate`를 수행하여 최종 답변을 생성합니다.   

In [ ]:
def decide_to_generate(state: GraphState):
    # 평가된 문서를 기반으로 다음 단계를 결정
    print("===Decide to Generate===")
    # 웹 검색 필요 여부
    web_search = state["web_search"]

    if web_search == "yes":
        # 웹 검색으로 정보 보강이 필요한 경우
        print("=== all documents are irrelevant ===")
        return "query_rewrite"
    else:
        # 관련 문서가 존재하므로 답변 생성 단계(generate)로 진행
        print("=== some documents are relevant ===")
        return "generate"
    
    

## 그래프 생성

노드를 정의하고 엣지를 연결하여 그래프를 완성합니다.

In [ ]:
from langgraph.graph import END, StateGraph, START

# 그래프 상태 초기화 
workflow = StateGraph(GraphState)

# 노드 정의
workflow.add_node("retrieve", retrieve)
workflow.add_node("grade_documents", grade_documents)
workflow.add_node("generage", generate)
workflow.add_node("query_rewrite", query_rewrite)
workflow.add_node("web_search", web_search)

# 엣지 연결 
workflow.add_edge(START,"retrieve")
workflow.add_edge("retrieve", "grade_documents")

# 문서 평가 노드에서 조건부 엣지 추가 
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "query_rewrite" : "query_rewrite",
        "generate" : "generate",
    },
)

# 엣지 연결
workflow.add_edge("query_rewrite", "web_search")
workflow.add_edge("web_search", "generate")
workflow.add_edge("generate", END)

# 그래프 컴파일
app = workflow.compile()


In [ ]:
from config.graphs import visualize_graph

visualize_graph(app)

## 그래프 실행 

In [ ]:
from langchian_core.runnables import RunnableConfig 
from config.messages import stream_graph, invoke_graph, random_uuid 

# config 설정 (재귀 최대 횟수, thread_id)
config = RunnableConfig(recursion_limit=20, configurable={"thread_id": random_uuid()})

# 질문 입력 
inputs = {"question": "삼성전자가 개발한 생성AI에 대해 설명하세요"}

# 스트리밍 형식으로 그래프 실행 
stream_graph(
    app,
    inputs, 
    config,
    ["retrieve", "grade_documents", "generate", "query_rewrite", "web_search", "generate"]
)

In [ ]:
# config 설정 (재귀 최대 횟수, thread_id)
config = RunnableConfig(recursion_limit=20, configurable={"thread_id":random_uuid()}) 

# 질문 입력
inputs = {"question" : "2024년 노벨 문학상 수상자의 이름은?"}

# 그래프 실행
invoke_graph(app, inputs, config)

In [ ]:
# 그래프 실행
stream_graph(
    app,
    inputs,
    config,
    ["retrieve", "grade_documents", "query_rewrite", "generate"]
)